In [ ]:
import cv2
import numpy as np
import pandas as pd
from skimage.measure import shannon_entropy

# Funciones para extraer características
def clarity_contrast(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 1, ksize=3)
    magnitude = np.sqrt(sobel**2)
    return np.mean(magnitude)

def hue_count(image, beta=0.05):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue = hsv[:, :, 0]
    hist = cv2.calcHist([hue], [0], None, [20], [0, 256])
    hist = hist.flatten()
    m = np.max(hist)
    Nc = np.sum(hist > beta * m)
    return 20 - Nc

def brightness_contrast(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist = hist.flatten()
    brightness_diff = np.max(hist) - np.min(hist)
    return brightness_diff

def color_entropy(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    entropy = 0
    for channel in cv2.split(lab):
        entropy += shannon_entropy(channel)
    return entropy

def composition_geometry(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    M = cv2.moments(edges)
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    h, w = edges.shape
    center_x, center_y = w // 2, h // 2
    return np.sqrt((cx - center_x)**2 + (cy - center_y)**2)

def background_simplicity(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    background_area = gray[thresh == 255]
    return np.std(background_area)

# Función para extraer características de una imagen
def extract_image_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"No se pudo cargar la imagen en {image_path}")
    
    features = {
        "image_path": image_path,
        "clarity_contrast": clarity_contrast(image),
        "hue_count": hue_count(image),
        "brightness_contrast": brightness_contrast(image),
        "color_entropy": color_entropy(image),
        "composition_geometry": composition_geometry(image),
        "background_simplicity": background_simplicity(image)
    }
    
    return features

# Función para procesar las imágenes y guardar las características en un CSV
def process_images_and_save_to_csv(image_csv, output_csv):
    # Leer el CSV que contiene las rutas de las imágenes
    df_paths = pd.read_csv(image_csv)
    
    
    df_paths = df_paths.iloc[2:]
    
    # Crear una lista para almacenar los resultados
    results = []
    
    # Iterar sobre las filas del DataFrame
    for _, row in df_paths.iterrows():
            # Extraer la ruta de la imagen
        image_path = f"../imgs/{row.values[0]}"
        features = extract_image_features(image_path)
        results.append(features)
    
    # Convertir los resultados a un DataFrame y guardarlo como CSV
    df_features = pd.DataFrame(results)
    df_features.to_csv(output_csv, index=False)
    print(f"Datos guardados en {output_csv}")

# Rutas de entrada y salida
paths_csv = "../X_test_paths.csv"  # Asegúrate de que sea un archivo CSV con una columna 'paths'
output_csv = "../X_test_features.csv"

# Procesar las imágenes y guardar las características en el CSV
# process_images_and_save_to_csv(paths_csv, output_csv)

/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/numpy/_core/_methods.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/numpy/_core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/numpy/_core/_methods.py:219: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ZeroDivisionError: float division by zero